In [ ]:
#Install first run only

!pip install weaviate-client
!pip install weaviate-client python-docx PyPDF2


In [1]:
# Connect to the weaviate server on Mac Studio
import os
import weaviate
import docx
# Bạn sẽ cần cài đặt thư viện bổ sung để xử lý tệp PDF, ví dụ: PyPDF2 hoặc pdfminer.six

# Chi tiết về địa chỉ Weaviate
weaviate_url = "http://192.168.1.86:8080"


# The following code embeds txt and doc files into the Weaviate server with the class name 'Documents' and properties 'content' and 'document_name'.
# It embeds files selected from the data folder, which is located in the same folder as this ipynb file.

In [2]:

try:
    weaviate_client = weaviate.Client(url=weaviate_url)

    # Định nghĩa schema cho tài liệu của bạn
    schema = {
        "classes": [{
            "class": "Document",
            "properties": [{
                "name": "text",
                "dataType": ["text"],
                "vectorizer": "text2vec-contextionary"
            }, {
                "name": "filename",
                "dataType": ["string"]
            }]
        }]
    }
    weaviate_client.schema.create(schema)

except Exception as e:
    print(f"Không thể kết nối đến Weaviate. Vui lòng kiểm tra URL và kết nối mạng của bạn. Lỗi: {e}")
    exit()

# Hàm trích xuất văn bản từ tệp
def extract_text_from_file(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == ".txt":
        with open(file_path, "r", encoding="utf-8") as file:
            return file.read()
    elif file_extension == ".docx":
        doc = docx.Document(file_path)
        full_text = [para.text for para in doc.paragraphs]
        return '\n'.join(full_text)
    elif file_extension == ".pdf":
        # Thực hiện logic trích xuất văn bản từ tệp PDF ở đây
        return "Nội dung từ tệp pdf"
    else:
        print(f"Loại tệp không được hỗ trợ: {file_extension}")
        return None

# Hàm nhúng tài liệu vào Weaviate
def embed_files_in_weaviate(file_paths):
    for file_path in file_paths:
        try:
            file_content = extract_text_from_file(file_path)
            if file_content is None:
                continue

            # Tạo một đối tượng dữ liệu
            data_object = {
                "text": file_content,
                "filename": os.path.basename(file_path)
            }

            # Xác định tên lớp (cập nhật theo schema của bạn trong Weaviate)
            class_name = "Document"

            # Thêm đối tượng dữ liệu vào Weaviate
            weaviate_client.data_object.create(data_object, class_name=class_name)

            print(f"Tệp {file_path} đã được nhúng thành công.")

        except Exception as e:
            print(f"Lỗi khi nhúng tệp {file_path}: {e}")

# Chỉ định đường dẫn thư mục chứa dữ liệu
data_folder_path = "data"  # Cập nhật với đường dẫn thư mục dữ liệu của bạn

# Lấy danh sách tất cả các tệp trong thư mục dữ liệu
file_paths_to_embed = [os.path.join(data_folder_path, file) for file in os.listdir(data_folder_path) if os.path.isfile(os.path.join(data_folder_path, file))]

# Nhúng các tệp vào Weaviate
embed_files_in_weaviate(file_paths_to_embed)


Loại tệp không được hỗ trợ: 
Tệp data/CV_cat_dien_12_2023.docx đã được nhúng thành công.
Tệp data/Ho_Chi_Minh.txt đã được nhúng thành công.


------------- The code below has the purpose of deleting the class created in the previous step ----------------------

In [1]:
# Connect to the Weaviate server on Mac Studio. Skip this step if you have already done so.

import weaviate

# Weaviate instance details
weaviate_url = "http://192.168.1.86:8080"
weaviate_client = weaviate.Client(url=weaviate_url)

In [2]:
# Specify the class name you want to delete. Note that the class name has to be created beforehand.
class_name = "Document"

In [4]:

# Retrieve and delete all objects of the specified class
try:
    # Retrieve all objects in the class
    response = weaviate_client.data_object.get(
        class_name=class_name, 
        limit=100  # Adjust the limit based on your expected number of objects
    )

    # Check if 'objects' key is in response
    if 'objects' in response:
        objects = response['objects']

        # Iterate and delete each object using its UUID
        for obj in objects:
            uuid = obj["id"]
            weaviate_client.data_object.delete(class_name=class_name, uuid=uuid)

        print(f"All objects of class {class_name} deleted successfully.")
    else:
        print("No 'objects' key found in response.")
except Exception as e:
    print(f"Error retrieving or deleting objects of class {class_name}: {e}")

# Delete the class itself
try:
    weaviate_client.schema.delete_class(class_name)
    print(f"Class {class_name} deleted successfully.")
except Exception as e:
    print(f"Error deleting class {class_name}: {e}")


Error retrieving or deleting objects of class Document: argument of type 'NoneType' is not iterable
Class Document deleted successfully.
